一些预处理，主要是让eta_schedule中没有空行.

In [1]:
import numpy as np
import pandas as pd

task_df = pd.read_csv("../database/data/eta_task.csv")
schedule_df = pd.read_csv("../database/data/eta_task_schedule.csv")

In [17]:
for i in range(len(schedule_df)):
    if pd.isnull(schedule_df.iloc[i]['path']):
        schedule_df.iloc[i,1] = str([(-1,-1)])
        schedule_df.iloc[i,2] = str([])
        schedule_df.iloc[i,3] = str([])
schedule_df.to_csv("../database/data/eta_schedule_noempty.csv")

In [ ]:
schedule_df = pd.read_csv("../database/data/eta_schedule_noempty.csv")
for r in schedule_df.iterrows():
    print(r[1][2])

进行预测

In [1]:
import numpy as np
import pandas as pd
import torch
from Dataset import ETATaskData, RoadFeatures
from GRU import GRUmodel, GRUBagging
from Utils import *

In [3]:
nbags = 6   # 筛选出来6个

trajSet = ETATaskData("../database/data/eta_schedule_noempty.csv", "../database/data/eta_task.csv", 8)
rawroadfeat = RoadFeatures("./road_features_with_lengths.pkl", "../database/data/road.csv")
base_bagging_dir = "./GRUBagging"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

GRUs = GRUBagging(rawroadfeat.n_features, device, nbags, is_training=False, params_path=base_bagging_dir).to(device)
fundGRU = GRUmodel(rawroadfeat.n_features, 128, device).eval().to(device)
fundGRU.load_state_dict(torch.load("./GRUonly_result/rawfeature_reg_gru.pt"))

ret = np.zeros((len(trajSet),2))
l = 0
for tids, roadids, tids, start_end_points, start_speeds in trajSet.iter_by_order():
    fund = fundGRU.predict(roadids, start_end_points, start_speeds, rawroadfeat)
    offset = GRUs.predict(roadids, start_speeds, rawroadfeat)
    t = torch.clamp(fund + offset, 2).detach().cpu().numpy()
    ret[l:l+t.shape[0], 0] = tids
    ret[l:l+t.shape[0], 1] = t
    l += t.shape[0]

df = pd.DataFrame(ret, columns=['tid', 'timd'])
df.to_csv("../database/data/eta_answer.csv")

d:\classes\Datamining\BUAA-DM23\ETA\Dataset.py:136: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.road_ids = [ast.literal_eval(r[1][2]) for r in schedule_df.iterrows()]   # 注意iterrows()的结果是一个元组，第一项是idx!
d:\classes\Datamining\BUAA-DM23\ETA\Dataset.py:138: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  paths = [ast.literal_eval(r[1][1]) for r in schedule_df.iterrows()]
